In [1]:
import numpy as np
from Reader_functions import *

In [2]:
with open("Nck-Nephrin-NWasp.txt", "r") as f:
    lines = f.readlines()

split_file = []
current_list = []

for line in lines:
    if line[0:3] == '***':
        split_file.append(current_list)
        current_list = []
    current_list.append(line.strip())
split_file.append(current_list)
split_file.pop(0)

for item in split_file:
    print(item)


['*** TIME INFORMATION ***', 'Total time: 5.00E-1', 'dt: 1.00E-8', 'dt_spring: 1.00E-9', 'dt_data: 2.00E-3', 'dt_image: 1.00E-4', '']
['*** SYSTEM INFORMATION ***', 'L_x: 0.1', 'L_y: 0.1', 'L_z_out: 0.01', 'L_z_in: 0.1', 'Partition Nx: 10', 'Partition Ny: 10', 'Partition Nz: 10', '']
['*** MOLECULES ***', '', 'MOLECULE: "NWASP" Intracellular Number 10 Site_Types 5 Total_Sites 16 Total_Links 15 is2D false', '{', 'TYPE: Name "dom1" Radius 2.00000 D 2.000 Color ORANGE STATES "State0"', 'TYPE: Name "linker" Radius 1.00000 D 2.000 Color PINK STATES "State0"', 'TYPE: Name "PRM" Radius 1.00000 D 2.000 Color GREEN STATES "State0"', 'TYPE: Name "dom3" Radius 2.25000 D 2.000 Color ORANGE STATES "State0"', 'TYPE: Name "dom2" Radius 1.60000 D 2.000 Color ORANGE STATES "State0"', '', "SITE 0 : Intracellular : Initial State 'State0'", 'TYPE: Name "PRM" Radius 1.00000 D 2.000 Color GREEN STATES "State0"', 'x 0.00000 y 34.00000 z 52.00000', "SITE 1 : Intracellular : Initial State 'State0'", 'TYPE: Nam

In [3]:
#Extract molecule data
molecules = []
molecule = []
for line in split_file[2]:
    if line[0:9] == 'MOLECULE:':
        molecules.append(molecule)
        molecule = []
    molecule.append(line)
molecules.append(molecule)    
molecules.pop(0)

for item in molecules:
    print(item)

['MOLECULE: "NWASP" Intracellular Number 10 Site_Types 5 Total_Sites 16 Total_Links 15 is2D false', '{', 'TYPE: Name "dom1" Radius 2.00000 D 2.000 Color ORANGE STATES "State0"', 'TYPE: Name "linker" Radius 1.00000 D 2.000 Color PINK STATES "State0"', 'TYPE: Name "PRM" Radius 1.00000 D 2.000 Color GREEN STATES "State0"', 'TYPE: Name "dom3" Radius 2.25000 D 2.000 Color ORANGE STATES "State0"', 'TYPE: Name "dom2" Radius 1.60000 D 2.000 Color ORANGE STATES "State0"', '', "SITE 0 : Intracellular : Initial State 'State0'", 'TYPE: Name "PRM" Radius 1.00000 D 2.000 Color GREEN STATES "State0"', 'x 0.00000 y 34.00000 z 52.00000', "SITE 1 : Intracellular : Initial State 'State0'", 'TYPE: Name "linker" Radius 1.00000 D 2.000 Color PINK STATES "State0"', 'x 0.00000 y 34.00000 z 55.50000', "SITE 2 : Intracellular : Initial State 'State0'", 'TYPE: Name "PRM" Radius 1.00000 D 2.000 Color GREEN STATES "State0"', 'x 0.00000 y 34.00000 z 59.00000', "SITE 3 : Intracellular : Initial State 'State0'", 'TYP

In [19]:
#Molecule Processing
for item in molecules:   
    #General Info
    typStr = ''
    split_general_info = item[0].split()
    site_types = int(split_general_info[6])
    for j in range(2,2+site_types):
        type_split = item[j].split()
        type_name = type_split[2]
        states = [i for i in type_split if '"' in i]
        states.pop(0)
        typStrAddition = ''
        if len(states) == 1:
            pass
        else:
            for state in states:
                typStrAddition = typStrAddition + f'{state[1:-1]}, '
            typStrAddition = typStrAddition[:-2]
            typStrAddition = f' ({len(states)} possible states: {typStrAddition})'
        typStr = f'{typStr}{type_name[1:-1]}{typStrAddition}, '
    print(f'MOLECULE: {split_general_info[1]}\n')
    print(f'This molecule has {split_general_info[6]} site types: {typStr[:-2]}')
        
    #Sites
    total_sites = int(split_general_info[8])
    site_data = []
    for i in range(3 + site_types, 3 + site_types + 3*total_sites, 3):
        site_split1 = item[i].split()
        site_split2 = item[i+1].split()
        site = [f'{site_split1[0]} {site_split1[1]}', site_split1[3], site_split2[2][1:-1]]
        site_data.append(site)

    sites_reordered = []
    site_type_names = []  
    for site in site_data:
        if site[2] not in site_type_names:
            site_type_names.append(site[2])
            #Site type name, Numpy Array(#Intracellular, #Extracellular, #Membrane), [list of sites] 
            sites_reordered.append([site[2], location2vec(site[1]), [site[0]]])
        else:
            for reordered_site in sites_reordered:
                if site[2] == reordered_site[0]:
                    reordered_site[1] = reordered_site[1] + location2vec(site[1])
                    reordered_site[2].append(site[0])

    #Sites (default ordering)
    print(f'\nThis molecule consists of {total_sites} connected sites:')
    for site in site_data:
        print(f'{site[0]} ({site[1]}) of type "{site[2]}"')
        pass

    #Sites (grouped by type)
    '''
    print(f'\nThis molecule consists of {total_sites} connected sites:')
    for reordered_site in sites_reordered:
        str2 = 'Placeholder 2'
        location_list = list(reordered_site[1])
        site_list = list(reordered_site[2])
        for index in location_list:
            addStr = ''
            if location_list[0] != 0:
                addStr = addStr + f'{location_list[0]} intracellular, '
            if location_list[1] != 0:
                addStr = addStr + f'{location_list[1]} extracellular, '
            if location_list[2] != 0:
                addStr = addStr + f'{location_list[2]} membrane, '
        addStr2 = ''
        for index in site_list:
            addStr2 = addStr2 + index + ', '
        print(f'Type {reordered_site[0]}: {addStr[:-2]} sites ({addStr2[:-2]})')
    '''
    
    total_links = int(split_general_info[10])
    link_data = []
    for i in range(4 + site_types + 3*total_sites, 4 + site_types + 3*total_sites + total_links):
        split_link = item[i].split()
        link = [split_link[2], split_link[5]]
        link_data.append(link)
    
    #[[link index, list of links],[link index, list of links],...]
    link_lists = []
    for site in site_data:
        split_site = site[0].split()
        link_count_init = [split_site[1], []]
        link_lists.append(link_count_init)
    for link_list in link_lists:
        for link in link_data:
            if link_list[0] == link[0]:
                link_list[1].append(link[1])
            elif link_list[0] == link[1]:
                link_list[1].append(link[0])
            else:
                pass

    link_lists_remaining = link_lists

    #Links (no repeated entries)
    '''print(f'\nThis molecule had {total_links} total links between sites:')
    while True:
        record_length = 0
        for link_list in link_lists:
            if len(link_list[1]) > record_length:
                record_length = len(link_list[1])
                record_loc = int(link_list[0])
        if record_length == 0:
            break

        site_str = ''
        for site in link_lists_remaining[record_loc][1]:
            site_str = f'{site_str} site {site},'
        
        print(f'Site {record_loc} is connected to{site_str[:-1]}')
        for link_list_remaining in link_lists_remaining:
            link_list_remaining[1] = [i for i in link_list_remaining[1] if i != str(record_loc)] 
        link_lists_remaining[record_loc][1] = []
    print('\n')'''

    #All links for all sites
    print(f'\nThis molecule had {total_links} total links between sites:')
    for link in link_lists:
        site_str = ''
        for site in link[1]:
            site_str = f'{site_str} site {site},'
        print(f'Site {link[0]} is connected to{site_str[:-1]}')


MOLECULE: "NWASP"

This molecule has 5 site types: dom1, linker, PRM, dom3, dom2

This molecule consists of 16 connected sites:
SITE 0 (Intracellular) of type "PRM"
SITE 1 (Intracellular) of type "linker"
SITE 2 (Intracellular) of type "PRM"
SITE 3 (Intracellular) of type "linker"
SITE 4 (Intracellular) of type "PRM"
SITE 5 (Intracellular) of type "linker"
SITE 6 (Intracellular) of type "PRM"
SITE 7 (Intracellular) of type "linker"
SITE 8 (Intracellular) of type "PRM"
SITE 9 (Intracellular) of type "linker"
SITE 10 (Intracellular) of type "PRM"
SITE 11 (Intracellular) of type "dom1"
SITE 12 (Intracellular) of type "dom3"
SITE 13 (Intracellular) of type "dom2"
SITE 14 (Intracellular) of type "linker"
SITE 15 (Intracellular) of type "linker"

This molecule had 15 total links between sites:
Site 0 is connected to site 13, site 1
Site 1 is connected to site 0, site 2
Site 2 is connected to site 1, site 3
Site 3 is connected to site 2, site 4
Site 4 is connected to site 3, site 5
Site 5 is 

In [29]:
#Reaction Rules
#Site R of molecules R (in any state) can bind to site A of adapter (in any state).
#Extract reaction data
transition_reactions = []
allosteric_reactions = []
binding_reactions = []
reaction_types = [['transition_reactions', 5], ['allosteric_reactions', 6], ['binding_reactions', 7]]
for reaction_type in reaction_types:
    for line in split_file[reaction_type[1]]:
        if line != '':
            exec(reaction_type[0] + '.append(line)')
    exec(reaction_type[0] + '.pop(0)')
    

'''
    molecule.append(line)
molecules.append(molecule)    
molecules.pop(0)

for item in molecules:
    print(item)
'''        



["'pY_SH2 binding_1'       'NEPHRIN' : 'pTyr_1_2' : 'p'  +  'NCK' : 'SH2' : 'Any_State'  kon 100.0  koff 100.0  Bond_Length 1.0", "'PRM_SH3_1'       'NCK' : 'SH3_1' : 'Any_State'  +  'NWASP' : 'PRM' : 'Any_State'  kon 100.0  koff 10000.0  Bond_Length 1.0", "'PRM_SH3_2'       'NCK' : 'SH3_2' : 'Any_State'  +  'NWASP' : 'PRM' : 'Any_State'  kon 100.0  koff 10000.0  Bond_Length 1.0", "'PRM_SH3_3'       'NCK' : 'SH3_3' : 'Any_State'  +  'NWASP' : 'PRM' : 'Any_State'  kon 100.0  koff 10000.0  Bond_Length 1.0", "'pY_SH2_binding_2'       'NEPHRIN' : 'pTyr_3' : 'p'  +  'NCK' : 'SH2' : 'Any_State'  kon 100.0  koff 100.0  Bond_Length 1.0"]


'\n    molecule.append(line)\nmolecules.append(molecule)    \nmolecules.pop(0)\n\nfor item in molecules:\n    print(item)\n'